# Play Cart Pole

In [ ]:
import gym
env = gym.make("CartPole-v0") #https://gym.openai.com/envs/#board_game

In [ ]:
print(env.observation_space.low, env.observation_space.high)
#cart position/velocity + pole angle/velocity at tip

In [ ]:
env.action_space.n  #http://gym.openai.com/docs/

In [ ]:
# Play randomly
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample()) # take a random action

env.close()

# Q Training

In [ ]:
import importlib
import qlearning
importlib.reload(qlearning)
import numpy as np
import gym
env = gym.make("CartPole-v0") #https://gym.openai.com/envs/#board_game

def state_function(state):
    if abs(state[0]) >= 2.2:
        x_pos = round(state[0], 2)
    else:
        x_pos = 0
    return (x_pos, round(state[1], 1), round(state[2], 2), qlearning.round_to(state[3], 1))
        
game = qlearning.GamePlayer(env, state_function)

In [ ]:
total_episodes = 1000
alpha = 0.3
gamma = 0.9                 # Discounting rate
decay_rate = 0.0          # Exponential decay rate for exploration prob
epsilon = 0.3                 # Exploration rate
#game.erase_training()
rewards = game.train(total_episodes, alpha, gamma, epsilon, decay_rate, logEvery=100)
print("Total reward average:", np.mean(rewards))
print(len(game.qtable))

In [ ]:
qlearning.visualize_computer_playing(5, game)

In [ ]:
game.qtable

# Double Q training

In [ ]:
import importlib
import qlearning
importlib.reload(qlearning)
import numpy as np
import gym
env = gym.make("CartPole-v0") #https://gym.openai.com/envs/#board_game

def state_function(state):
    if abs(state[0]) >= 2.2:
        x_pos = round(state[0], 2)
    else:
        x_pos = 0
    return (x_pos, round(state[1], 1), round(state[2], 2), qlearning.round_to(state[3], 1))
        
game = qlearning.GamePlayer(env, state_function)

In [ ]:
total_episodes = 50
alpha = 0.3
gamma = 0.9                 # Discounting rate
decay_rate = 0.0          # Exponential decay rate for exploration prob
epsilon = 0.3                 # Exploration rate
#game.erase_training()
rewards = game.double_q_train(total_episodes, alpha, gamma, epsilon, decay_rate, logEvery=10)
print("Total reward average:", np.mean(rewards))
print(len(game.qtable))

In [ ]:
qlearning.visualize_computer_playing(5, game, True)

In [ ]:
print(game.Q2)

# Using tensorflow

In [ ]:
import tensorflow as tf
import gym
import numpy as np
from qlearning import round_to
env = gym.make("CartPole-v0") #https://gym.openai.com/envs/#board_game

In [ ]:
env.observation_space.shape[0]

In [ ]:
state_dim = env.observation_space.shape[0]

tf.reset_default_graph()
alpha = 0.8
epsilon = 0.2
gamma = 0.9
N = 30 # 10: n steps before losing when playing with a non trained network
total_episodes = 160
inputs = tf.placeholder(shape=[1,state_dim],dtype=tf.float32)
training_inputs = tf.placeholder(shape=[N,state_dim],dtype=tf.float32)
states = tf.Variable(tf.zeros([N,state_dim],dtype=tf.float32))
ytarget = tf.Variable(tf.zeros([N,env.action_space.n],dtype=tf.float32))
W = tf.Variable(tf.random_uniform([state_dim, env.action_space.n],0,0.01))
theta_target = {}
theta = {}
Qs = tf.matmul(states, W)
Q = tf.matmul(inputs, W)

loss = tf.reduce_sum(tf.square(ytarget - Qs))
trainer = tf.train.GradientDescentOptimizer(learning_rate=alpha)
updateModel = trainer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
reward_list = []
tot_reward_list = []
logEvery = N

with tf.Session() as sess:
    sess.run(init)
    nstep = 0
    for episode in range(total_episodes):
        state = env.reset()
        done = False
        tot_reward = 0
        while done is False:
            if nstep == N:
                sess.run([updateModel,W])
                nstep = 0

            states[nstep].assign(state)

            Y = sess.run([Q], feed_dict={inputs: [state]})[0][0]
            if np.random.rand(1) < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(Y)
            
            next_state, reward, done, _ = env.step(action)
            
            if done:
                Y[action] = reward
            else:
                Y[action] = reward + gamma * np.max(sess.run([Q], feed_dict={inputs: [next_state]}))
            ytarget[nstep].assign(Y)
            
            
            state = next_state
            tot_reward += reward 
            nstep += 1
            reward_list.append(tot_reward)

        if logEvery > 0 and (episode+1) % logEvery == 0:
            ave_reward = np.mean(reward_list)
            tot_reward_list.append(ave_reward)
            reward_list = []
            print('Episode {} Average Reward: {}, alpha: {}, e: {}'.format(episode+1, ave_reward, alpha, epsilon))
print("Total reward average:", np.mean(tot_reward_list))